In [1]:
import os
import pandas as pd
import numpy as np
import ast
import json

# Organisation

In [2]:
DATA_DIR = os.getenv("DATA_DIR")
org_edge_list = os.path.join(DATA_DIR, "org_edgelist_from_api.csv")

In [53]:
df = pd.read_csv(org_edge_list)

In [54]:
df.head()

,src_base_path,src_cid,src_title,target_base_path,target_cid,target_title,link_type,src_orgid,target_orgid
0,academy-for-social-justice,b854f170-53c8-4098-bf77-e8ef42f93107,Academy for Social Justice,ministry-of-justice,dcc907d6-433c-42df-9ffb-d9c68be5dc4d,Ministry of Justice,has_parent_org,OT1276,D18
1,academy-for-social-justice,b854f170-53c8-4098-bf77-e8ef42f93107,Academy for Social Justice,academy-for-social-justice-commissioning,ce357bdb-6396-426a-9f1f-8cbfb444cffd,Academy for Social Justice Commissioning,has_superseded_org,OT1276,OT1208
2,accelerated-access-review,a0f338c5-e94c-42f8-9c26-b9c2eb6850d3,Accelerated Access Review,department-of-health-and-social-care,7cd6bf12-bbe9-4118-8523-f927b0442156,Department of Health and Social Care,has_parent_org,OT1137,D12
3,administration-of-radioactive-substances-advis...,b5bd9a64-4315-492e-9679-3e2f6799c769,Administration of Radioactive Substances Advis...,department-of-health-and-social-care,7cd6bf12-bbe9-4118-8523-f927b0442156,Department of Health and Social Care,has_parent_org,PB523,D12
4,administrative-court,b0bdfcf3-2763-4002-961e-a0b2d7825038,Administrative Court,hm-courts-and-tribunals-service,6f757605-ab8f-4b62-84e4-99f79cf085c2,HM Courts & Tribunals Service,has_parent_org,CO1188,EA73


## Org

In [10]:
# creating dict removes duplicates, merge the two sets
src_dict = dict(zip(df.src_title, df.src_orgid))
target_dict = dict(zip(df.target_title, df.target_orgid))
# https://stackoverflow.com/questions/38987/how-to-merge-two-dictionaries-in-a-single-expression
org_dict = {**src_dict, **target_dict}

In [47]:
# this contrasts to the complete set in the Registers API which has almost 1k rows
len(org_dict)

763

In [29]:
org_nodes = pd.DataFrame.from_dict(org_dict, orient="index")

org_nodes.reset_index(inplace = True)
org_nodes.columns = ['title', 'orgid']
org_nodes.head()

,title,orgid
0,Academy for Social Justice,OT1276
1,Accelerated Access Review,OT1137
2,Administration of Radioactive Substances Advis...,PB523
3,Administrative Court,CO1188
4,Admiralty Court,CO1147


In [30]:
org_nodes.to_csv(os.path.join(DATA_DIR, "Org.csv"), index=False)

## OrgOrgRel


In [55]:
# can use this, but might be worth relabelling
# for an Organisation
df.head()

,src_base_path,src_cid,src_title,target_base_path,target_cid,target_title,link_type,src_orgid,target_orgid
0,academy-for-social-justice,b854f170-53c8-4098-bf77-e8ef42f93107,Academy for Social Justice,ministry-of-justice,dcc907d6-433c-42df-9ffb-d9c68be5dc4d,Ministry of Justice,has_parent_org,OT1276,D18
1,academy-for-social-justice,b854f170-53c8-4098-bf77-e8ef42f93107,Academy for Social Justice,academy-for-social-justice-commissioning,ce357bdb-6396-426a-9f1f-8cbfb444cffd,Academy for Social Justice Commissioning,has_superseded_org,OT1276,OT1208
2,accelerated-access-review,a0f338c5-e94c-42f8-9c26-b9c2eb6850d3,Accelerated Access Review,department-of-health-and-social-care,7cd6bf12-bbe9-4118-8523-f927b0442156,Department of Health and Social Care,has_parent_org,OT1137,D12
3,administration-of-radioactive-substances-advis...,b5bd9a64-4315-492e-9679-3e2f6799c769,Administration of Radioactive Substances Advis...,department-of-health-and-social-care,7cd6bf12-bbe9-4118-8523-f927b0442156,Department of Health and Social Care,has_parent_org,PB523,D12
4,administrative-court,b0bdfcf3-2763-4002-961e-a0b2d7825038,Administrative Court,hm-courts-and-tribunals-service,6f757605-ab8f-4b62-84e4-99f79cf085c2,HM Courts & Tribunals Service,has_parent_org,CO1188,EA73


Need to filter by the different link_type and create a separate csv for each one.

In [57]:
df.link_type.value_counts()

has_parent_org         650
has_child_org          641
has_superseding_org    183
has_superseded_org     178
Name: link_type, dtype: int64

In [62]:
def create_org_rel(relationship, df = df):
    """Filter the link_type column by relationship type. Write edgelist to csv into DATA_DIR."""
    edgelist = df[df.link_type.str.contains(relationship)]
    edgelist.to_csv(os.path.join(DATA_DIR, ('org_'+relationship+'.csv')), index=False)

We assume child and superseded are of the most interest - no point including symmetrical relationships.

In [63]:
create_org_rel('has_child_org')
create_org_rel('has_superseded_org')

# Cid

In [139]:
DATA_DIR = os.getenv("DATA_DIR")
content = os.path.join(DATA_DIR, "preprocessed_content_store.csv")

In [140]:
df = pd.read_csv(content)

In [141]:
# create dictionary from string
df["orgs_id"] =  df["orgs_id"].map(lambda x: ast.literal_eval(x) if not isinstance(x, float) else {}
)
df["orgs_title"] =  df["orgs_title"].map(lambda x: ast.literal_eval(x) if not isinstance(x, float) else {}
)

## Cid

In [67]:
df.head(2)

,base_path,content_id,title,description,document_type,orgs_id,orgs_title,text
0,/1619-bursary-fund,f4b96a38-5247-4afd-b554-8a258a0e8c93,16 to 19 Bursary Fund,"Bursaries of up to £1,200 for students in furt...",guide,{'organisations': ['71381a6e-aa5c-43ae-a982-be...,{'organisations': ['Education and Skills Fundi...,You could get a bursary to help with education...
1,/30-hours-free-childcare,ddda6dc8-e9de-49db-bbd1-97e3d0bc1e6f,30 hours free childcare,Who is eligible for 30 hours free childcare an...,answer,{'organisations': ['ebd15ade-73b2-4eaf-b1c3-43...,"{'organisations': ['Department for Education',...",You may be able to get up to 30 hours free chi...


In [41]:
print(len(df.content_id.unique()))
print(len(df.base_path.unique()))
df.shape

344736
360921


(360921, 8)

While base_paths are the unique id within content store, having duplicate cids is not that common. I would force it to be a one to one mapping for now, since once we query and want to return something, i'd prefer it if it was all unique.

In [68]:
df_cid = df.groupby(['content_id'])['base_path'].apply(list).reset_index()

#change column order                           
df_cid.head()

,content_id,base_path
0,0000d0a0-037a-4110-a271-24327f422d06,[/government/news/new-digital-resource-for-cha...
1,0000e128-f9d5-4115-8203-b892fcd90044,[/guidance/237867b9-e889-45c2-ae47-0e22857a160e]
2,00012147-49f6-4e90-be1f-e50bb719f53d,[/government/publications/patent-journal-speci...
3,00015d3f-e7d9-48e8-95ff-ac3f7fa07be3,[/government/statistics/uk-consumer-price-infl...
4,00019715-9eec-4bef-b447-bca558dbabbb,[/aaib-reports/piper-pa-28-161-cherokee-warrio...


Simple lists can be stored as attributes of nodes in Neo4j. Look at the attribute [array in this example](https://neo4j.com/docs/cypher-manual/current/functions/list/).

In [88]:
# roll up so that one cid per row - as cid is our node, entity not base_path
# simple lists can be stored as attributes of nodes in Neo4j
# https://neo4j.com/docs/cypher-manual/current/functions/list/
df_cid = df.groupby('content_id').agg({ 'base_path': lambda x: list(x),
                                       'title': lambda x: list(x),
                                       'description': lambda x: list(x),
                                      'document_type': lambda x: list(x),
                                      'orgs_id': lambda x: list(x),
                                      'orgs_title': lambda x: list(x),
                                      'text': lambda x: list(x)})

In [89]:
df_cid.head()

,base_path,title,description,document_type,orgs_id,orgs_title,text
content_id,,,,,,,
0000d0a0-037a-4110-a271-24327f422d06,[/government/news/new-digital-resource-for-cha...,[New digital resource for charity trustees lau...,"[The tool, developed by the Commission, Grant ...",[press_release],[{'organisations': ['489e651f-34c8-4b34-bdd7-e...,"[{'organisations': ['The Charity Commission'],...",[The Charity Commission Grant Thornton and Zoe...
0000e128-f9d5-4115-8203-b892fcd90044,[/guidance/237867b9-e889-45c2-ae47-0e22857a160e],[()],[()],[detailed_guide],[{'organisations': ['af07d5a5-df63-4ddc-9383-6...,[{'organisations': ['Government Digital Servic...,[nan]
00012147-49f6-4e90-be1f-e50bb719f53d,[/government/publications/patent-journal-speci...,[Patent Journal special notices: 6636],[Publication date 27 July 2016.],[notice],[{'organisations': ['5d6f9583-991f-413d-ae83-b...,[{'organisations': ['Intellectual Property Off...,[These are notices which appear for a limited ...
00015d3f-e7d9-48e8-95ff-ac3f7fa07be3,[/government/statistics/uk-consumer-price-infl...,[UK consumer price inflation: Dec 2017],"[Price indices, percentage changes and weights...",[national_statistics],[{'organisations': ['5da4ab94-39b2-40cf-99ae-1...,[{'organisations': ['Office for National Stati...,[Official statistics are produced impartially ...
00019715-9eec-4bef-b447-bca558dbabbb,[/aaib-reports/piper-pa-28-161-cherokee-warrio...,"[Piper PA-28-161 Cherokee Warrior II, G-BUJO, ...",[nan],[aaib_report],[{'organisations': ['38eb5d8f-2d89-480c-8655-e...,[{'organisations': ['Air Accidents Investigati...,[Piper PA-28-161 Cherokee Warrior II G-BUJO Do...


In [90]:
# we add an attribute to each content_id node whether it is unique or not (1 for unique)
df_cid['base_path_count'] = df_cid.base_path.str.len()

In [91]:
df_cid.base_path_count.value_counts()

1     333484
2       6888
3       4090
5        172
4         68
6         12
8          9
7          8
20         1
15         1
10         1
9          1
Name: base_path_count, dtype: int64

In [92]:
df_cid.shape

(344735, 8)

In [93]:
# make content_id a column
df_cid.reset_index(level=0, inplace=True)

BUG / ISSUE - some of the entries are `[nan]` or `'[()]'`. This should be replaced with Cypher `null` (maybe). Might also be able to fix in Cypher by MATCH and SET.


In [95]:
# write to tsv as base_path can have commas
df_cid.to_csv(os.path.join(DATA_DIR, ('Cid.csv')), sep='\t', index=False)

## CidOrgRel from Content Store data
We want information from the content store about how different Organsiations relate to each piece of content. We note that each dictionary is a list of varying length as sometimes these values are missing, as per our analysis which revealed:

*Name: organisations*
327,934 content items include tag
95.94% include tag

*Name: primary_publishing_organisation*
262,582 content items include tag
76.82% include tag

*Name: original_primary_publishing_organisation*
50,743 content items include tag
14.84% include tag

*Name: worldwide_organisations*
12,213 content items include tag
3.57% include tag

*Name: supporting_organisations*
69 content items include tag
0.02% include tag

In [193]:
pd.set_option('max_colwidth',500)

df.orgs_title.head(3)

0                                                        {'organisations': ['Education and Skills Funding Agency'], 'primary_publishing_organisation': ['Government Digital Service']}
1    {'organisations': ['Department for Education', 'Department for Education and Skills', 'HM Revenue & Customs'], 'primary_publishing_organisation': ['Government Digital Service']}
2                                                 {'organisations': ['Air Accidents Investigation Branch'], 'primary_publishing_organisation': ['Air Accidents Investigation Branch']}
Name: orgs_title, dtype: object

In [ ]:
# split into different cols, need to paste a name on it otherwise they overwrite
df_split = pd.concat([df.drop(['orgs_title'], axis=1), df['orgs_title'].apply(pd.Series)], axis=1)
# df_split = pd.concat([df.drop(['orgs_id'], axis=1), df['orgs_id'].apply(pd.Series)], axis=1)

In [161]:
df_split.head()

,base_path,content_id,title,description,document_type,orgs_id,text,organisations,primary_publishing_organisation,worldwide_organisations,original_primary_publishing_organisation,supporting_organisations
0,/1619-bursary-fund,f4b96a38-5247-4afd-b554-8a258a0e8c93,16 to 19 Bursary Fund,"Bursaries of up to £1,200 for students in further education, training or unpaid apprenticeships - what’s available, eligibility and how to apply",guide,"{'organisations': ['71381a6e-aa5c-43ae-a982-be9bfd46ea5b'], 'primary_publishing_organisation': ['af07d5a5-df63-4ddc-9383-6a666845ebe9']}",You could get a bursary to help with education-related costs if you’re aged 16 to 19 and: studying at a publicly funded school or college in England - not a university on a training course including unpaid work experience A publicly funded school is one that doesn’t charge you for attending it. There’s a different scheme in Wales Scotland and Northern Ireland . If you’re 19 and over You could also get a bursary if you either: are continuing on a course you started aged 16 to 18 (known as bei...,[Education and Skills Funding Agency],[Government Digital Service],NaN,NaN,NaN
1,/30-hours-free-childcare,ddda6dc8-e9de-49db-bbd1-97e3d0bc1e6f,30 hours free childcare,Who is eligible for 30 hours free childcare and how you can get it.,answer,"{'organisations': ['ebd15ade-73b2-4eaf-b1c3-43034a42eb37', '77c1621f-a392-4393-9d8c-9969cd98c1e7', '6667cce2-e809-4e21-ae09-cb0bdc1ddda3'], 'primary_publishing_organisation': ['af07d5a5-df63-4ddc-9383-6a666845ebe9']}",You may be able to get up to 30 hours free childcare (1 140 hours per year which you can choose how you take) if your child is 3 to 4 years old. The childcare: must be with an approved childcare provider stops when your child starts in reception class (or reaches compulsory school age if later) There are different schemes in Scotland Wales and Northern Ireland . If you’re eligible for the extra hours you sign up online to get a code to give to your childcare provider to reserve your place. Y...,"[Department for Education, Department for Education and Skills, HM Revenue & Customs]",[Government Digital Service],NaN,NaN,NaN
2,/aaib-reports/1-1971-g-atek-and-g-ateh-15-august-1967,ed760821-bf95-408b-9824-f6efccd1b505,"1/1971 G-ATEK and G-ATEH, 15 August 1967",NaN,aaib_report,"{'organisations': ['38eb5d8f-2d89-480c-8655-e2e7ac23f8f4'], 'primary_publishing_organisation': ['38eb5d8f-2d89-480c-8655-e2e7ac23f8f4']}",Report No: 1/1971. Hawker Siddeley HS 748 Series 2 Aircraft G-ATEK and G-ATEH of Channel Airways. Report on the accidents at Portsmouth Airport Portsmouth Hampshire on 15 August 1967 Download report: 1-1971 G-ATEK and G-ATEH.pdf (2 918.68 kb) Report Appendices To view appendices click on link below: 1/1971 G-ATEK and G-ATEH Append (598.34 kb),[Air Accidents Investigation Branch],[Air Accidents Investigation Branch],NaN,NaN,NaN
3,/aaib-reports/1-1973-ph-moa-3-june-1971,bd9737e8-44fe-4928-985c-803b5fa7ad9f,"1/1973 PH-MOA, 3 June 1971",NaN,aaib_report,"{'organisations': ['38eb5d8f-2d89-480c-8655-e2e7ac23f8f4'], 'primary_publishing_organisation': ['38eb5d8f-2d89-480c-8655-e2e7ac23f8f4']}",Report No: 1/1973. Douglas DC3 PH-MOA. Report on the accident at Southend Airport on 3 June 1971 Download report: 1-1973 PH-MOA.pdf (1 101.96 kb),[Air Accidents Investigation Branch],[Air Accidents Investigation Branch],NaN,NaN,NaN
4,/aaib-reports/1-1975-beechcraft-95-b55-baron-g-azzj-4-january-1974,36a0e60e-de72-4374-98fe-c201fdb86068,"1/1975 Beechcraft 95-B55 (Baron), G-AZZJ, 4 January 1974",NaN,aaib_report,"{'organisations': ['38eb5d8f-2d89-480c-8655-e2e7ac23f8f4'], 'primary_publishing_organisation': ['38eb5d8f-2d89-480c-8655-e2e7ac23f8f4']}",Report No: 1/1975. Beechcraft 95-B55 (Baron) G-AZZJ. Report on the accident at Cholesbury cum St Leonards Buckinghamshire on 4 January 1974 Download report: 1-1975 G-AZZJ.pdf (1 939.20 kb),[Air Accidents Investigation Branch],[Air Accidents Investigation Branch],NaN,NaN,NaN


### cid_has_organisations_org
Need to get the edge list in the form...
With the `organisations` lists expanded so you have one row per relationship. Then drop `NaN` or non-existent relationships.

src_cid, target_title  

In [229]:
# https://stackoverflow.com/questions/13413590/how-to-drop-rows-of-pandas-dataframe-whose-value-in-certain-columns-is-nan
# get just the cols we need and drop NaN ~5%
cid_has_organisations_org = df_split[['content_id', 'organisations']].dropna(subset = ['organisations']).copy()

In [230]:
print(cid_has_organisations_org.shape)
cid_has_organisations_org.head()

(334776, 2)


,content_id,organisations
0,f4b96a38-5247-4afd-b554-8a258a0e8c93,[Education and Skills Funding Agency]
1,ddda6dc8-e9de-49db-bbd1-97e3d0bc1e6f,"[Department for Education, Department for Education and Skills, HM Revenue & Customs]"
2,ed760821-bf95-408b-9824-f6efccd1b505,[Air Accidents Investigation Branch]
3,bd9737e8-44fe-4928-985c-803b5fa7ad9f,[Air Accidents Investigation Branch]
4,36a0e60e-de72-4374-98fe-c201fdb86068,[Air Accidents Investigation Branch]


In [231]:
# https://stackoverflow.com/questions/27263805/pandas-when-cell-contents-are-lists-create-a-row-for-each-element-in-the-list/48532692#48532692

def explode(df, lst_col):
    """Explode a dataframe by expanding a list column."""

    r = pd.DataFrame({
      col:np.repeat(df[col].values, df[lst_col].str.len())
      for col in df.columns.drop(lst_col)}
    ).assign(**{lst_col: np.concatenate(df[lst_col].values)})[df.columns]
    
    return r

In [232]:
cid_has_organisations_org = explode(df = cid_has_organisations_org, lst_col = 'organisations')

In [233]:
print(cid_has_organisations_org.shape)
cid_has_organisations_org.head()
# no NaNs found
# no empty lists found

(395055, 2)


,content_id,organisations
0,f4b96a38-5247-4afd-b554-8a258a0e8c93,Education and Skills Funding Agency
1,ddda6dc8-e9de-49db-bbd1-97e3d0bc1e6f,Department for Education
2,ddda6dc8-e9de-49db-bbd1-97e3d0bc1e6f,Department for Education and Skills
3,ddda6dc8-e9de-49db-bbd1-97e3d0bc1e6f,HM Revenue & Customs
4,ed760821-bf95-408b-9824-f6efccd1b505,Air Accidents Investigation Branch


Let's use the `org_dict` to lookup the `org_id` of our org names. This presumably will be faster when writing to the database as it's faster to match. 

In [234]:
# Our analytics_identifier is missing some worldwide organisations
cid_has_organisations_org.assign(orgid=cid_has_organisations_org['organisations'].map(org_dict)).tail()
# let's drop these for now as they aren't associated with much content

,content_id,organisations,orgid
395050,5f557e7f-7631-11e4-a3cb-005056011aef,Department for International Trade Italy,NaN
395051,5f557577-7631-11e4-a3cb-005056011aef,UK Mission to the UN in Vienna,NaN
395052,de09348e-cb0c-4d58-96f3-4ff2156a236e,The Insolvency Service,EA32
395053,290292c3-0ad9-48a0-aebc-0d255e3425ba,Youth Justice Board for England and Wales,PB302
395054,0dae8b02-356e-46ae-8c42-b3e979777589,"Department of Agriculture, Environment and Rural Affairs (Northern Ireland)",OT1178


In [235]:
cid_has_organisations_org = cid_has_organisations_org.assign(orgid=cid_has_organisations_org['organisations'].map(org_dict))

In [236]:
cid_has_organisations_org = cid_has_organisations_org[pd.notnull(cid_has_organisations_org['orgid'])]
cid_has_organisations_org.shape

(386636, 3)

In [237]:
# write to tsv for consistency
cid_has_organisations_org.to_csv(os.path.join(DATA_DIR, ('cid_has_organisations_org.csv')), sep='\t', index=False)

### cid_has_primary_publishing_organisation_org


In [251]:
def exploder(df, lst_col, org_dict):
    """Explode a relationship type col from a dataframe and save the edgelist as tsv.
    
    Use the org_dict to lookup org_id by org name, allows faster writing to database."""
    # https://stackoverflow.com/questions/13413590/how-to-drop-rows-of-pandas-dataframe-whose-value-in-certain-columns-is-nan
    df = df[['content_id', lst_col]].dropna(subset = [lst_col])
    
    # https://stackoverflow.com/questions/27263805/pandas-when-cell-contents-are-lists-create-a-row-for-each-element-in-the-list/48532692#48532692
    r = pd.DataFrame({
      col:np.repeat(df[col].values, df[lst_col].str.len())
      for col in df.columns.drop(lst_col)}
    ).assign(**{lst_col: np.concatenate(df[lst_col].values)})[df.columns]
    
    print("The number of edges", r.shape)
    
    # Get org id assoicated with the name of the Government Organisation
    r = r.assign(orgid=r[lst_col].map(org_dict))
    # Drop Organisations for which we don't have an ID
    # This might drop all worldwide organisations relationships
    r = r[pd.notnull(r['orgid'])]

    print("The number of edges not including NaNs (no corresponding org_id)", r.shape)

    r.to_csv(os.path.join(DATA_DIR, ("cid_has_"+lst_col+"_org"+".csv")), sep='\t', index=False)
    return r.head()
    

In [252]:
exploder(df = df_split, lst_col = 'primary_publishing_organisation', org_dict = org_dict)

The number of edges (269316, 2)
The number of edges not including NaNs (no corresponding org_id) (261901, 3)


,content_id,primary_publishing_organisation,orgid
0,f4b96a38-5247-4afd-b554-8a258a0e8c93,Government Digital Service,OT1056
1,ddda6dc8-e9de-49db-bbd1-97e3d0bc1e6f,Government Digital Service,OT1056
2,ed760821-bf95-408b-9824-f6efccd1b505,Air Accidents Investigation Branch,OT248
3,bd9737e8-44fe-4928-985c-803b5fa7ad9f,Air Accidents Investigation Branch,OT248
4,36a0e60e-de72-4374-98fe-c201fdb86068,Air Accidents Investigation Branch,OT248


### cid_has_worldwide_organisations_org


In [253]:
# We need a new type of entity, "Worldwide organisation"
# It's inherently different from Organisation, as 
exploder(df = df_split, lst_col = 'worldwide_organisations', org_dict = org_dict)

The number of edges (27848, 2)
The number of edges not including NaNs (no corresponding org_id) (0, 3)


,content_id,worldwide_organisations,orgid


### cid_has_original_primary_publishing_organisation_org

In [254]:
exploder(df = df_split, lst_col = 'original_primary_publishing_organisation', org_dict = org_dict)

The number of edges (52387, 2)
The number of edges not including NaNs (no corresponding org_id) (50886, 3)


,content_id,original_primary_publishing_organisation,orgid
0,2ec3aa83-e6b2-482d-8b86-d7c8e8bfd2b8,"Department for Digital, Culture, Media & Sport",D5
1,78c06002-19cb-44b2-8bd3-6bfaf3e11c17,"Department for Digital, Culture, Media & Sport",D5
2,b93a455b-a4ab-4eed-87b3-d5ea89d800d4,"Department for Digital, Culture, Media & Sport",D5
3,5ff3f2c2-1222-47d7-b0dc-7909dfaa919f,"Department for Digital, Culture, Media & Sport",D5
4,3cec363f-8778-4cf9-9823-9f2169f67a89,Crown Commercial Service,EA1015


### cid_has_supporting_organisations_org

In [255]:
exploder(df = df_split, lst_col = 'supporting_organisations', org_dict = org_dict)

The number of edges (72, 2)
The number of edges not including NaNs (no corresponding org_id) (72, 3)


,content_id,supporting_organisations,orgid
0,5e94a61f-7631-11e4-a3cb-005056011aef,"Department for Business, Innovation & Skills",D3
1,5e94a410-7631-11e4-a3cb-005056011aef,"Department for Business, Innovation & Skills",D3
2,5e94a565-7631-11e4-a3cb-005056011aef,"Department for Business, Innovation & Skills",D3
3,5e94a3c8-7631-11e4-a3cb-005056011aef,"Department for Business, Innovation & Skills",D3
4,d8db7b61-c560-4705-9ad0-057919c702c6,Foreign & Commonwealth Office,D13


## CidOrgRel not from Content Store
Graph databases are useful when we join up different datasets. We can use NLP to enhance our understanding of the text associated with each piece of content by searching for organisations, as the Content Store data might be incomplete.

## CidCidRel not from Content Store
As a data driven organisation we continue to accrue data through our data science and machine learning pipelines. We incorporate a subset of that data into our knowledge graph.

Specifcally we use the new feature: `suggested_ordered_related_items` predicted with the node2vec algorithm trained on the functional and structural network of GOV.UK. For the Related links repo, see [here](https://github.com/alphagov/govuk-related-links-recommender).

In [4]:
related_links = os.path.join(DATA_DIR, "20190607top100_suggested_related_links.csv")
df = pd.read_csv(related_links)
# we didn't read in as tsv, doesn't seem to be a problem

In [7]:
print(df.shape)
df.tail()
# already one row per edge

(500, 5)


,target_content_id,probability,source_content_id,source_base_path,target_base_path
495,3f83e66f-633c-4795-a8ea-008250466c55,0.953494,c8b5f020-6620-478e-997d-bfabd7ec91dc,/government/publications/income-tax-repayment-...,/government/publications/flexibly-accessed-pen...
496,60273b1b-7631-11e4-a3cb-005056011aef,0.912956,c8b5f020-6620-478e-997d-bfabd7ec91dc,/government/publications/income-tax-repayment-...,/government/publications/income-tax-claiming-t...
497,60273a31-7631-11e4-a3cb-005056011aef,0.902923,c8b5f020-6620-478e-997d-bfabd7ec91dc,/government/publications/income-tax-repayment-...,/government/collections/income-tax-forms
498,f9ebc063-7210-49a3-84ce-f86f253097fc,0.890580,c8b5f020-6620-478e-997d-bfabd7ec91dc,/government/publications/income-tax-repayment-...,/government/publications/income-tax-claim-for-...
499,60273ea9-7631-11e4-a3cb-005056011aef,0.855353,c8b5f020-6620-478e-997d-bfabd7ec91dc,/government/publications/income-tax-repayment-...,/government/publications/income-tax-claim-for-...


Turns out the edge list is already of the required shape with one row per relationship or edge. We can rewrite this as taread this straight into neo4j with Cypher.

In [8]:
df.to_csv(os.path.join(DATA_DIR, ("cid_has_"+"suggested_ordered_related_items"+"_cid"+".csv")), sep='\t', index=False)